In [1]:
import pandas as pd

In [2]:
df_sentiment = pd.read_csv('../data/final_dataset/sentiment_df.csv')

In [5]:
df_sentiment.head(3)

,index,id,title,text,dateCreate,product,sub_product,source,text_clean,review,predicted_classes_refined,final_classes,num_tokens,sentiment
0,0,992651,не выполняют условия акции,в апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,Умная карта (Премиум),sravni,в апреле года я рекомендовала дебетовую карту ...,не выполняют условия акции в апреле года я ре...,"[1, 3, 14, 15]","['debitCards', 'cashbackPromo', 'notifications...",262,NEGATIVE
1,1,998360,жалоба на услугу газпром бонус премиум,купил услугу газпром бонус премиум за 2 цель б...,2025-09-15T09:38:13.34818Z,debitCards,Отсутствует,sravni,купил услугу газпром бонус премиум за цель был...,жалоба на услугу газпром бонус премиум купил ...,[],['debitCards'],155,NEGATIVE
2,2,993744,банк не отвечает за слова своих сотрудников не...,хочу поделиться историей которая убила моё дов...,2025-09-02T23:21:16.507166Z,debitCards,Умная карта,sravni,хочу поделиться историей которая моё к газпром...,банк не отвечает за слова своих сотрудников не...,"[1, 6, 15]","['debitCards', 'cashbackPromo', 'earlyRepaymen...",309,NEGATIVE


In [6]:
df_sentiment = df_sentiment.drop(columns = ['predicted_classes_refined', 'num_tokens', 'review', 'text_clean'])

In [7]:
df_sentiment.head(3)

,index,id,title,text,dateCreate,product,sub_product,source,final_classes,sentiment
0,0,992651,не выполняют условия акции,в апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,Умная карта (Премиум),sravni,"['debitCards', 'cashbackPromo', 'notifications...",NEGATIVE
1,1,998360,жалоба на услугу газпром бонус премиум,купил услугу газпром бонус премиум за 2 цель б...,2025-09-15T09:38:13.34818Z,debitCards,Отсутствует,sravni,['debitCards'],NEGATIVE
2,2,993744,банк не отвечает за слова своих сотрудников не...,хочу поделиться историей которая убила моё дов...,2025-09-02T23:21:16.507166Z,debitCards,Умная карта,sravni,"['debitCards', 'cashbackPromo', 'earlyRepaymen...",NEGATIVE


In [8]:
df_classes = pd.read_csv('../data/final_dataset/classification_df.csv')

In [9]:
df_classes.head(3)

,index,text,final_classes,predicted_classes
0,0,в апреле 2025 года я рекомендовала дебетовую к...,"['debitCards', 'cashbackPromo', 'notifications...","('cashbackPromo', 'debitCards', 'notifications..."
1,1,купил услугу газпром бонус премиум за 2 цель б...,['debitCards'],"('debitCards',)"
2,2,хочу поделиться историей которая убила моё дов...,"['debitCards', 'cashbackPromo', 'earlyRepaymen...","('cashbackPromo', 'debitCards', 'earlyRepaymen..."


In [10]:
df_classes.shape

(50357, 4)

In [11]:
df_sentiment.shape

(50357, 10)

In [34]:
df_final = df_sentiment.merge(df_classes[['index', 'predicted_classes']], on = 'index', how = 'inner')

In [35]:
df_final.sample(3)

,index,id,title,text,dateCreate,product,sub_product,source,final_classes,sentiment,predicted_classes
6147,6147,12406925,акция приведи друга не работает,в начале июля 2025 года я отправил жене ссылку...,2025-07-12 12:59:18,debitCards,Отсутствует,banki,['debitCards'],NEGATIVE,"('debitCards',)"
3434,3434,904281,я потерял 45 тр по халатности сотрудника гпб,г железногорск курская область сотрудницы банк...,2024-12-01T20:00:30.469119Z,savings,Отсутствует,sravni,['savings'],POSITIVE,"('savings',)"
3963,3963,394982,рефинансирование,здравствуйте имею кредит на сумму хотел сделат...,2021-02-17T20:41:44.566786Z,restructing,Отсутствует,sravni,['restructing'],NEUTRAL,"('restructing',)"


In [37]:
from dateutil import parser
from tqdm import tqdm

tqdm.pandas()

def parse_date_safe(x):
    if pd.isna(x):
        return pd.NaT
    try:
        # убираем Z, если есть
        x = x.replace("Z", "")
        return parser.parse(x)
    except Exception:
        return pd.NaT

In [38]:
# применяем к колонке с прогрессбаром
df_final["dateCreate_dt"] = df_final["dateCreate"].progress_apply(parse_date_safe)

# делаем только дату
df_final["dateCreate"] = df_final["dateCreate_dt"].dt.date

# YEAR и MONTH
df_final["YEAR"] = df_final["dateCreate_dt"].dt.year
df_final["MONTH"] = df_final["dateCreate_dt"].dt.month


100%|██████████| 50357/50357 [00:00<00:00, 59309.60it/s]


In [42]:
df_final

,index,id,title,text,dateCreate,product,sub_product,source,final_classes,sentiment,predicted_classes,dateCreate_dt,YEAR,MONTH
0,0,992651,не выполняют условия акции,в апреле 2025 года я рекомендовала дебетовую к...,2025-08-29,debitCards,Умная карта (Премиум),sravni,"['debitCards', 'cashbackPromo', 'notifications...",NEGATIVE,"('cashbackPromo', 'debitCards', 'notifications...",2025-08-29 23:30:38.746003,2025,8
1,1,998360,жалоба на услугу газпром бонус премиум,купил услугу газпром бонус премиум за 2 цель б...,2025-09-15,debitCards,Отсутствует,sravni,['debitCards'],NEGATIVE,"('debitCards',)",2025-09-15 09:38:13.348180,2025,9
2,2,993744,банк не отвечает за слова своих сотрудников не...,хочу поделиться историей которая убила моё дов...,2025-09-02,debitCards,Умная карта,sravni,"['debitCards', 'cashbackPromo', 'earlyRepaymen...",NEGATIVE,"('cashbackPromo', 'debitCards', 'earlyRepaymen...",2025-09-02 23:21:16.507166,2025,9
3,3,999494,не выполняют условия акции,в июне 2025 года я порекомендовал премиальную ...,2025-09-18,debitCards,Отсутствует,sravni,"['debitCards', 'cashbackPromo', 'notifications...",NEGATIVE,"('cashbackPromo', 'debitCards', 'notifications...",2025-09-18 08:10:19.954986,2025,9
4,4,992291,странный банк,добрый день в связи с устройством на новую раб...,2025-08-28,debitCards,Отсутствует,sravni,"['debitCards', 'depositAccess', 'remoteService']",NEGATIVE,"('debitCards', 'depositAccess', 'remoteService')",2025-08-28 19:29:51.696341,2025,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50352,50352,10885124,под видом кредита 12 9 пытаются впарить кредит...,газпромбанк тут заманивал кредитом одобренным ...,2023-04-15,individual,Отсутствует,banki,"['other', 'earlyRepayment', 'loanIssues']",NEGATIVE,"('credits', 'earlyRepayment', 'loanIssues')",2023-04-15 23:33:45.000000,2023,4
50353,50353,10872392,неполное изменение номера в офисе отделении,ситуация довольно странная для входа в интерне...,2023-03-30,individual,Отсутствует,banki,['other'],NEGATIVE,(),2023-03-30 12:39:01.000000,2023,3
50354,50354,10852603,один из отличных банков,в банке нравится все и обслуживание на отлично...,2023-03-10,individual,Отсутствует,banki,['other'],POSITIVE,"('debitCards',)",2023-03-10 15:04:12.000000,2023,3
50355,50355,10852597,обращение в контакт центр 10 баллов,обращение в контакт центр 10 баллов а вот рабо...,2023-03-10,individual,Отсутствует,banki,['other'],POSITIVE,"('debitCards',)",2023-03-10 14:57:02.000000,2023,3


In [41]:
df_final['YEAR'].isna().sum()

np.int64(0)

# Clickhouse

In [88]:
import clickhouse_connect

# создаём клиент
client = clickhouse_connect.get_client(
    host='128.69.178.214',
    port=8123,
    username='ch_admin',
    password='ch_admin_password',
    database='default'
)


In [80]:
review_cols = ["index", "id", "source", "title", "text", "dateCreate", "YEAR", "MONTH", "product", "sub_product", "sentiment"]
df_review = df_final[review_cols].copy()

In [81]:
df_review

,index,id,source,title,text,dateCreate,YEAR,MONTH,product,sub_product,sentiment
0,0,992651,sravni,не выполняют условия акции,в апреле 2025 года я рекомендовала дебетовую к...,2025-08-29,2025,8,debitCards,Умная карта (Премиум),NEGATIVE
1,1,998360,sravni,жалоба на услугу газпром бонус премиум,купил услугу газпром бонус премиум за 2 цель б...,2025-09-15,2025,9,debitCards,Отсутствует,NEGATIVE
2,2,993744,sravni,банк не отвечает за слова своих сотрудников не...,хочу поделиться историей которая убила моё дов...,2025-09-02,2025,9,debitCards,Умная карта,NEGATIVE
3,3,999494,sravni,не выполняют условия акции,в июне 2025 года я порекомендовал премиальную ...,2025-09-18,2025,9,debitCards,Отсутствует,NEGATIVE
4,4,992291,sravni,странный банк,добрый день в связи с устройством на новую раб...,2025-08-28,2025,8,debitCards,Отсутствует,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...
50352,50352,10885124,banki,под видом кредита 12 9 пытаются впарить кредит...,газпромбанк тут заманивал кредитом одобренным ...,2023-04-15,2023,4,individual,Отсутствует,NEGATIVE
50353,50353,10872392,banki,неполное изменение номера в офисе отделении,ситуация довольно странная для входа в интерне...,2023-03-30,2023,3,individual,Отсутствует,NEGATIVE
50354,50354,10852603,banki,один из отличных банков,в банке нравится все и обслуживание на отлично...,2023-03-10,2023,3,individual,Отсутствует,POSITIVE
50355,50355,10852597,banki,обращение в контакт центр 10 баллов,обращение в контакт центр 10 баллов а вот рабо...,2023-03-10,2023,3,individual,Отсутствует,POSITIVE


In [92]:
client.command('''
CREATE TABLE IF NOT EXISTS review (
    idx UInt32,
    id UInt64,
    source String,
    title String,
    review_text String,
    dateCreate Date,
    YEAR UInt16,
    MONTH UInt8,
    product String,
    sub_product String,
    sentiment String
) ENGINE = MergeTree()
ORDER BY id
''')

df_review = df_review.rename(columns={'index': 'idx', 'text': 'review_text'})

In [94]:
str_cols = ['source', 'title', 'review_text', 'product', 'sub_product', 'sentiment']
for col in str_cols:
    df_review[col] = df_review[col].fillna('').astype(str)

# Приведение даты к типу datetime.date (ClickHouse Date)
df_review['dateCreate'] = pd.to_datetime(df_review['dateCreate']).dt.date

In [96]:
client.insert_df('review', df_review)

In [101]:
import ast

# Если в колонке class строки вида "['a','b']", сначала преобразуем в список
df_classes['class'] = df_classes['class'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Разворачиваем списки по строкам
df_classes_exploded = df_classes.explode('class')

# Приведение NaN к пустой строке (на всякий случай)
df_classes_exploded['class'] = df_classes_exploded['class'].fillna('').astype(str)

In [102]:
df_classes_exploded

,id,class
0,992651,debitCards
0,992651,cashbackPromo
0,992651,notifications
0,992651,refinancing
0,992651,remoteService
...,...,...
50352,10885124,loanIssues
50353,10872392,other
50354,10852603,other
50355,10852597,other


In [104]:
client.command('''
CREATE TABLE IF NOT EXISTS review_classes (
    id UInt64,
    class String
) ENGINE = MergeTree()
ORDER BY id
''')

In [105]:
client.insert_df('review_classes', df_classes_exploded)

In [98]:
df_classes = df_final[["id", "final_classes"]].explode("final_classes")
df_classes.rename(columns={"final_classes": "class"}, inplace=True)

In [89]:
df_classes['class'] = df_classes['class'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [90]:
df_classes

,id,class
0,992651,"[debitCards, cashbackPromo, notifications, ref..."
1,998360,[debitCards]
2,993744,"[debitCards, cashbackPromo, earlyRepayment, re..."
3,999494,"[debitCards, cashbackPromo, notifications, ref..."
4,992291,"[debitCards, depositAccess, remoteService]"
...,...,...
50352,10885124,"[other, earlyRepayment, loanIssues]"
50353,10872392,[other]
50354,10852603,[other]
50355,10852597,[other]


In [69]:
client.command('''
CREATE TABLE IF NOT EXISTS review (
    idx UInt32,
    review_id UInt64,
    source String,
    title String,
    text String,
    dateCreate Date,
    year UInt16,
    month UInt8,
    product String,
    sub_product String,
    sentiment String
) ENGINE = MergeTree()
ORDER BY review_id
''')

In [70]:
client.command('''
CREATE TABLE IF NOT EXISTS review_classes (
    id UInt64,
    class String
) ENGINE = MergeTree()
ORDER BY id
''')

In [71]:
df_review = df_final.rename(columns={
    "index": "idx",
    "id": "review_id",
    "YEAR": "year",
    "MONTH": "month"
})[["idx","review_id","source","title","text","dateCreate","year","month","product","sub_product","sentiment"]]

df_review = df_review.fillna("").astype(str)

In [72]:
import ast

df_final['final_classes_list'] = df_final['final_classes'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

df_classes = df_final[['id','final_classes_list']].explode('final_classes_list')

df_classes = df_classes[df_classes['final_classes_list'] != '']

df_classes.rename(columns={'id':'review_id', 'final_classes_list':'class'}, inplace=True)

df_classes = df_classes.astype(str)

In [74]:
df_review_clean = df_review.rename(columns={
    'idx': 'index',
    'review_id': 'id',
    'year': 'YEAR',
    'month': 'MONTH'
})

# Заполняем NaN пустыми строками и приводим к строкам (для безопасности)
df_review_clean = df_review_clean.fillna("").astype(str)

# Преобразуем в список словарей
records = df_review_clean.to_dict('records')

# Проверим ключи словаря
print(records[0].keys())

dict_keys(['index', 'id', 'source', 'title', 'text', 'dateCreate', 'YEAR', 'MONTH', 'product', 'sub_product', 'sentiment'])


In [77]:
df_review_clean = df_review_clean.rename(columns={
    'idx': 'index',
    'review_id': 'id',
    'year': 'YEAR',
    'month': 'MONTH'
})

In [79]:
# 1️⃣ Приводим имена колонок к соответствию ClickHouse
df_review_clean = df_review_clean.rename(columns={
    'idx': 'index',
    'review_id': 'id',
    'year': 'YEAR',
    'month': 'MONTH'
})

# 2️⃣ Приводим все к строкам, убираем NaN
df_review_clean = df_review_clean.fillna("").astype(str)

# 3️⃣ Преобразуем DataFrame в список словарей (row-oriented)
records = df_review_clean.to_dict('records')

# 4️⃣ Вставляем в ClickHouse
client.insert(
    table='review',
    data=records,           # row-oriented
    column_oriented=False   # обязательно False для списка словарей
)


KeyError: 2

In [76]:
df_review_clean = df_review.fillna("").astype(str)

# Проверяем результат
print(df_review_clean.head())
print(df_review_clean.to_dict('list').keys())

# Вставка в ClickHouse
client.insert(
    table='review',
    data={col: df_review_clean[col].tolist() for col in df_review_clean.columns},  # column-oriented
    column_oriented=True
)

  idx review_id  source                                              title  \
0   0    992651  sravni                        не выполняют условия акции    
1   1    998360  sravni            жалоба на услугу газпром бонус премиум    
2   2    993744  sravni  банк не отвечает за слова своих сотрудников не...   
3   3    999494  sravni                        не выполняют условия акции    
4   4    992291  sravni                                      странный банк   

                                                text  dateCreate  year month  \
0  в апреле 2025 года я рекомендовала дебетовую к...  2025-08-29  2025     8   
1  купил услугу газпром бонус премиум за 2 цель б...  2025-09-15  2025     9   
2  хочу поделиться историей которая убила моё дов...  2025-09-02  2025     9   
3  в июне 2025 года я порекомендовал премиальную ...  2025-09-18  2025     9   
4  добрый день в связи с устройством на новую раб...  2025-08-28  2025     8   

      product            sub_product sentiment  
0

KeyError: 0

In [63]:
client.insert(table='review', data=df_review.to_dict('records'))
client.insert(table='review_classes', data=df_classes.to_dict('records'))

KeyError: 2

In [52]:
# ------------------------------
# Загрузка review
# ------------------------------
client.insert(
    table='review',
    data=df_review.to_dict('records')
)

# ------------------------------
# Загрузка review_classes
# ------------------------------
client.insert(
    table='review_classes',
    data=df_classes.to_dict('records')
)


KeyError: 2